In [ ]:
from rich import print

from example_results import result_with_chapters

transcript = result_with_chapters.parsed_transcript
print(transcript)

## Introduction
[Music] Hi, I'm Leela with NVIDIA, and this is the Jetson AGX Thor Developer Kit. Jetson Thor is the ultimate 
platform for humanoid robotics. It's part of NVIDIA's three-computer solution for accelerating physical AI: NVIDIA 
DGX for training, NVIDIA Omniverse for synthetic data generation and physical AI simulation, and NVIDIA Jetson Thor
for runtime robotics. Jetson AGX Thor is the ideal runtime computer for any kind of physical AI application, from 
humanoids to Edge AI agents. It gives you unmatched performance and scalability in a compact, power-efficient form 
factor. Plus, it's powered by the advanced Blackwell GPU and 128 GB of memory, delivering up to 2070 FP4 TFLOPS of 
AI compute to effortlessly run the latest generative AI models.

## What's in the Box
Let's take a closer look at the Jetson AGX Thor Developer Kit. It includes a Jetson T5000 module, a reference 
carrier board, an active heat sink with a fan, and a power supply. If you want to set up with a monitor, you'll 
need to provide a computer display, keyboard, and mouse to get started. To make your setup process as smooth as 
possible, we've conveniently

## Hardware Overview
placed all the I/O ports on one side of the board. This thoughtful design choice makes it super easy to connect 
your sensors and manage your wiring when building your robot. Starting here, we see two USB-A ports, an Ethernet 
port, a DisplayPort, an HDMI port, two USB-C ports, a QSFP slot, and a Micro-Fit port for extending the board's 
power if needed. Under the developer kit, we have an M.2 key M slot with a populated 1TB NVMe storage and an M.2 
key E slot with a pre-installed wireless networking card. On the other side of the developer kit, there are three 
buttons for power, force recovery, and reset. It's easy to get started. In the box is a

## First Power and Boot
small booklet that includes a link to step-by-step instructions. First, download the latest NVIDIA JetPack 7 ISO 
image from the JetPack SDK download page and flash it to a USB drive to install on your developer kit. After 
attaching your monitor, keyboard, and mouse, just connect the power supply and your developer kit will power on 
automatically. On first boot, you'll be prompted to choose a username, password, and other basic information. You 
can also set up your Wi-Fi at this time. Jetson Thor delivers a seamless cloud-to-edge

## The NVIDIA Software Stack
experience by running the NVIDIA AI software stack for physical AI applications. This includes NVIDIA Isaac for 
robotics, NVIDIA Metropolis for video analytics AI agents, and NVIDIA Holoscan for sensor processing. With up to 
7.5x the performance of Jetson AGX Orin, Jetson Thor can run all modern AI models from vision-language-action 
models like NVIDIA Isaac GR00T N1 to all popular LLMs and VLMs. NVIDIA Isaac GR00Tis redefining the future of 
robotics with four key pillars: robotic foundational models, synthetic data pipelines, a simulation environment, 
and a runtime computer. You can also accelerate your software development by tethering the developer kit directly 
to any existing robot. Here, NVIDIA Isaac GR00T N1 is running on Thor. To show a developer's end-to-end

## Isaac GR00T N1
journey, we've fine-tuned GR00T N1 for picking up a nut and pouring it in Isaac Sim and Isaac Lab. We've also 
implemented a hardware-in-the-loop scenario running NVIDIA Omniverse on RTX Pro 6000 and inferencing on Jetson 
Thor. The AI blueprint for Video Search and

## Video Search and Summarization
Summarization (VSS) from NVIDIA Metropolis gives you the tools to build and deploy video analytics AI agents that 
can perform contextualized real-time alerts, video summarization, and Q&A by analyzing live camera streams. VSS is 
powering visual agent workforces spanning many use cases, including visual inspection and worker safety in 
manufacturing, fan engagement and player analytics in live sports, and improved emergency response times for 
roadway incidents

In [6]:
"""
This module provides functions for processing transcribed text to generate formatted subtitles and AI-powered summaries using the Gemini API.
"""

import os

from dotenv import load_dotenv
from google.genai import Client, types
from pydantic import BaseModel, Field

from youtube_summarizer.utils import is_youtube_url

load_dotenv()


class Chapter(BaseModel):
    header: str = Field(description="A descriptive title for the chapter")
    key_points: list[str] = Field(description="Important takeaways and insights from this chapter")
    summary: str = Field(description="A comprehensive summary of the chapter content")


class Analysis(BaseModel):
    title: str = Field(description="The main title or topic of the video content")
    summary: str = Field(description="A comprehensive summary of the video content")
    takeaways: list[str] = Field(description="Key insights and actionable takeaways for the audience")
    key_facts: list[str] = Field(description="Important facts, statistics, or data points mentioned")
    chapters: list[Chapter] = Field(description="Structured breakdown of content into logical chapters")
    keywords: list[str] = Field(description="Keywords or topics mentioned in the video, max 3", max_length=3)


def summarize_video(url_or_transcript: str) -> Analysis:
    """
    Summarize the text using the Gemini. Streaming seems to be less buggy with long videos.
    """
    if is_youtube_url(url_or_transcript):
        print(f"🔗 Sending YouTube URL to Gemini: {url_or_transcript}")
    else:
        print(f"📝 Sending transcript text to Gemini: {len(url_or_transcript)} characters")
        print(f"📝 Text preview: {url_or_transcript[:200]}...")

    client = Client(
        api_key=os.getenv("GEMINI_API_KEY"),
        http_options={"timeout": 600000},
    )

    response = client.models.generate_content_stream(
        model="models/gemini-2.5-flash-lite",
        contents=types.Content(
            parts=[
                types.Part(file_data=types.FileData(file_uri=url_or_transcript)) if is_youtube_url(url_or_transcript) else types.Part(text=url_or_transcript),
            ]
        ),
        config=types.GenerateContentConfig(
            system_instruction="""Analyze the video/transcript according to the schema.
Consider the chapters (headers) if given but not necessary.
Ignore the promotional and meaningless content.
Follow the original language.""",
            temperature=0,
            response_mime_type="application/json",
            response_schema=Analysis,
            thinking_config=types.ThinkingConfig(thinking_budget=2048),
        ),
    )

    result_parts = [chunk.text for chunk in response if chunk.text is not None]
    final_result = "".join(result_parts)
    final_result = Analysis.model_validate_json(final_result)  # Convert to Pydantic model

    return final_result


result = summarize_video(transcript)
print(result)

📝 Sending transcript text to Gemini: 5149 characters

📝 Text preview: ## Introduction
[Music] Hi, I'm Leela with NVIDIA, and this is the Jetson AGX Thor Developer Kit. Jetson Thor is the ultimate 
platform for humanoid robotics. It's part of NVIDIA's three-computer solut...

Analysis(
    title='Jetson AGX Thor Developer Kit Overview',
    summary="The Jetson AGX Thor Developer Kit is presented as the ultimate platform for humanoid robotics and 
physical AI applications, complementing NVIDIA's DGX and Omniverse solutions. It offers unmatched performance and 
scalability with its Blackwell GPU, 128 GB memory, and up to 2070 FP4 TFLOPS of AI compute, capable of running the 
latest generative AI models. The kit includes the Jetson T5000 module, a carrier board, heat sink, and power 
supply, with user-friendly I/O placement. Setup involves flashing JetPack 7, connecting peripherals, and 
configuring the system. It runs the NVIDIA AI software stack, including Isaac for robotics, Metropolis for video 
analytics, and Holoscan for sensor processing, delivering up to 7.5x the performance of Jetson AGX Orin. Specific 
applications like NVIDIA Isaac GR00T N1 for foundational robotics models and Video Search and Summarization (VSS) 
for edge AI video analytics are highlighted, along with the Holoscan platform for real-time AI and sensor 
processing.",
    takeaways=[
        'Jetson AGX Thor is the premier runtime platform for humanoid robotics and physical AI.',
        "It leverages NVIDIA's comprehensive AI ecosystem (DGX, Omniverse, Jetson).",
        'The platform delivers high performance with the Blackwell GPU and 128GB memory, supporting advanced 
generative AI models.',
        'The developer kit is designed for ease of use with accessible I/O and straightforward setup.',
        "It integrates seamlessly with NVIDIA's software stack (Isaac, Metropolis, Holoscan) for diverse AI 
applications.",
        'It offers significant performance gains over previous Jetson generations.'
    ],
    key_facts=[
        'Powered by the advanced Blackwell GPU.',
        'Equipped with 128 GB of memory.',
        'Delivers up to 2070 FP4 TFLOPS of AI compute.',
        'Includes Jetson T5000 module, reference carrier board, active heat sink with fan, and power supply.',
        'Features USB-A, Ethernet, DisplayPort, HDMI, USB-C, QSFP, and Micro-Fit ports.',
        'Includes M.2 key M slot with 1TB NVMe storage and M.2 key E slot with wireless networking.',
        'Offers up to 7.5x the performance of Jetson AGX Orin.',
        'Supports NVIDIA Isaac GR00T N1, LLMs, and VLMs.',
        'NVIDIA Isaac GR00T N1 has four key pillars: foundational models, synthetic data pipelines, simulation, and
runtime.',
        'Video Search and Summarization (VSS) from NVIDIA Metropolis enables real-time video analytics.',
        'NVIDIA Holoscan simplifies and scales edge AI with high-performance sensor processing.'
    ],
    chapters=[
        Chapter(
            header='Introduction',
            key_points=[
                'Jetson AGX Thor is the ultimate platform for humanoid robotics and physical AI.',
                "It's part of NVIDIA's three-computer solution (DGX, Omniverse, Jetson).",
                'Powered by Blackwell GPU, 128GB memory, up to 2070 FP4 TFLOPS AI compute.',
                'Designed for runtime robotics and edge AI agents.'
            ],
            summary="This section introduces the Jetson AGX Thor Developer Kit as the premier platform for humanoid
robotics and physical AI, highlighting its role in NVIDIA's comprehensive AI ecosystem. It emphasizes the 
platform's powerful specifications, including the Blackwell GPU and substantial memory, enabling it to run advanced
generative AI models with high performance."
        ),
        Chapter(
            header="What's in the Box",
            key_points=[
                'The developer kit includes a Jetson T5000 module.',
                'It comes with a reference carrier board.',
                'An active heat sink with a fan is included.',
                'A power supply is provided.'
            ],
            summary='This part details the contents of the Jetson AGX Thor Developer Kit, listing the essential 
components required for setup